In [1]:
from os import path
import subprocess

In [11]:
def compile_and_program(project_path, device_name="ATSAML11E16A", verbose=0, clean=False, build=True, erase=True, program=True, verify=True, jobs=8, tool="EDBG", interface="SWD", atmel_studio_path="C:\\Program Files (x86)\\Atmel\\Studio", configuration="Debug", device_sn=None):
    # NOTE: if multiple devices are connected the serial number needs to be supplied to atprogram using the -s flag.
    # if path.split
    make_path = path.join(atmel_studio_path, "7.0\\shellutils\\make.exe")
    atprogram_path = path.join(atmel_studio_path, "7.0\\atbackend\\atprogram.exe")
    stdout = subprocess.PIPE if verbose else None
    if clean:
        res = subprocess.run([make_path, "clean"], cwd=path.join(project_path, configuration), stdout=stdout)
        if verbose: print(res.stdout.decode())
        if res.returncode: return res.returncode
    if build:
        res = subprocess.run([make_path, "all", "--jobs", str(jobs), "--output-sync"], cwd=path.join(project_path, configuration), stdout=stdout)
        if verbose: print(res.stdout.decode())
        if res.returncode: return res.returncode
    if erase or program or verify:
        atprogram_base_command = [atprogram_path] + ["-v"]*verbose + ["-t", tool, "-i", interface, "-d", device_name] + ["-s", device_sn]*(device_sn is not None)
    if erase:
        res = subprocess.run(atprogram_base_command + ["chiperase"], stdout=stdout)
        if verbose: print(res.stdout.decode())
        if res.returncode: return res.returncode
    if program or verify:
        if path.splitext(project_path)[1] is ".elf":
            elf_file_path = project_path
        else: 
            elf_file_path = path.join(project_path, configuration, path.split(project_path)[1]) + ".elf"
    if program:
        res = subprocess.run(atprogram_base_command + ["program", "-f", elf_file_path], stdout=stdout)
        if verbose: print(res.stdout.decode())
        if res.returncode: return res.returncode
    if verify:
        res = subprocess.run(atprogram_base_command + ["verify", "-f", elf_file_path], stdout=stdout)
        if verbose: print(res.stdout.decode())
#         if res.returncode: return res.returncode
    if verbose >= 2:
        res = subprocess.run(atprogram_base_command + ["info"], stdout=stdout)
        if verbose: print(res.stdout.decode())
        if res.returncode: return res.returncode
    return 0

In [12]:
project_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\pydgilib\\UnitTest\\UnitTest"

In [14]:
compile_and_program(project_path, verbose=1)

make: Nothing to be done for 'all'.

Firmware check OK
Chiperase completed successfully

Firmware check OK
Programming completed successfully.

Firmware check OK



0

```
>atprogram -d ATSAML11E16A -t EDBG -i SWD write --values 0x00
Firmware check OK
Write completed successfully.
```

```
>atprogram -d ATSAML11E16A -t EDBG -i SWD exitcode
Exit codes for atprogram

atprogram is the Atmel Studio Command Line Interface which is deployed with
Atmel Studio 6 as well as in a standalone package.  It replaces the individual
Tools' command line interfaces, including:
  - Stk500.exe
  - Stk600.exe
  - jtagiceii.exe
  - AVRDragon.exe
  - avrone.exe

atprogram also replaces the command line interface to UC3 devices- avr32program
atprogram also provides command line access to JTAGICE3 and SAMICE.

atprogram makes use of the Atmel Studio backend, which means that all tools
and devices supported by Atmel Studio are supported by atprogram.exe

This guide is intended to be used as a reference when using atprogram
in script utilities.

Code ranges:
 0-9         Exit codes that is locked or unknown
 10-50       Exit codes due to errors in atprogram itself
 100-200     Exit codes from atbackend


Exits Codes:

0    Success                       The program executed with no errors
1    Unexpected Error              A run time error that was not anticipated
2    User Interrupt Error          A user interrupted the program
10   Command Error                 Exception which indicates that a command has
                                   failed for some internal reason.
11   Command Argument Error        Raised when there are error in the arguments
                                   passed to the command
12   Missing Command Error         An error which occurs if there is no command
13   Command Parser Error          Error when parsing command line arguments.
14   Tool Error                    Exception raised when there is an error with
                                   the setup or use of a tool
15   Resource Error                Exception raised when there is an error in
                                   creating or serving a resource.
16   Tcf Error                     A TCF command returned an error.
17   Timeout Error                 Error because of an timeout.
18   Missing Or Wrong Input Error  An error caused by incomplete information
                                   specified by user
19   Old Fw Exception              Firmware must be upgraded
20   Backend Creation Error        Exception raised when there is an error in
                                   creating or serving a resource.
21   Options Parse Error           Error in parsing an option.
22   Json Parse Error              Exception which can be thrown if a TCF
                                   response could not be parsed.
23   Verify Error                  An error given when failing to verify memory
                                   content
```